In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re

from utils import *

In [2]:
df = pd.read_csv('../data/train.csv')
df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [3]:
im_count = df[df.Id != 'new_whale'].Id.value_counts()
im_count.name = 'sighting_count'
df = df.join(im_count, on='Id')
val_fns = set(df.sample(frac=1)[(df.Id != 'new_whale') & (df.sighting_count > 1)].groupby('Id').first().Image)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [5]:
pd.to_pickle(val_fns, '../data/val_fns.pkl')
val_fns = pd.read_pickle('../data/val_fns.pkl')

In [6]:
fn2label = {row[1].Image: row[1].Id for row in df.iterrows()}

In [7]:
SZ = 224
BS = 64
NUM_WORKERS = 0
SEED=0

In [8]:
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)

In [9]:
df = df[df.Id != 'new_whale']

In [10]:
df.shape

(15697, 3)

In [11]:
df.sighting_count.max()

73.0

In [12]:
df_val = df[df.Image.isin(val_fns)]
df_train = df[~df.Image.isin(val_fns)]
df_train_with_val = df

In [13]:
df_val.shape, df_train.shape, df_train_with_val.shape

((2931, 3), (12766, 3), (15697, 3))

In [14]:
%%time

res = None
sample_to = 15

for grp in df_train.groupby('Id'):
    n = grp[1].shape[0]
    additional_rows = grp[1].sample(0 if sample_to < n  else sample_to - n, replace=True)
    rows = pd.concat((grp[1], additional_rows))
    
    if res is None: res = rows
    else: res = pd.concat((res, rows))

CPU times: user 18.6 s, sys: 14.1 ms, total: 18.6 s
Wall time: 18.6 s


In [15]:
%%time

res_with_val = None
sample_to = 15

for grp in df_train_with_val.groupby('Id'):
    n = grp[1].shape[0]
    additional_rows = grp[1].sample(0 if sample_to < n  else sample_to - n, replace=True)
    rows = pd.concat((grp[1], additional_rows))
    
    if res_with_val is None: res_with_val = rows
    else: res_with_val = pd.concat((res_with_val, rows))

CPU times: user 18.9 s, sys: 15.6 ms, total: 18.9 s
Wall time: 18.9 s


In [16]:
res.shape, res_with_val.shape

((76174, 3), (76287, 3))

In [17]:
pd.concat((res, df_val))[['Image', 'Id']].to_csv('../data/oversampled_train.csv', index=False)
res_with_val[['Image', 'Id']].to_csv('../data/oversampled_train_and_val.csv', index=False)